## Big Query Machine Learning (BQML)
Learing Objectives
- Understand that it is possible to build ML models in Big Query
- Understand when this is appropriate
- Experience building a model using BQML

BigQuery is more than just a data warehouse, it also has some ML capabilities baked into it. 

As of January 2019 it is limited to linear models, but what it gives up in complexity, it gains in ease of use.

BQML is a great option when a linear model will suffice, or when you want a quick benchmark to beat, but for more complex models such as neural networks you will need to pull the data out of BigQuery and into an ML Framework like TensorFlow.

In this notebook, we will build a naive model using BQML. **This notebook is intended to inspire usage of BQML, we will not focus on model performance.**

## 1) Create BigQuery Dataset

Prior to now we've just been reading an existing BigQuery table, now we're going to create our own so so we need some place to put it. In BigQuery parlance, 'Dataset' means a folder for tables. 

We will take advantage of BigQuery's [Python Client](https://cloud.google.com/bigquery/docs/reference/libraries#client-libraries-install-python) to create the dataset.

In [ ]:
!pip install --upgrade google-cloud-bigquery

In [55]:
from google.cloud import bigquery
client = bigquery.Client()

dataset = bigquery.Dataset(client.dataset('bqml_taxifare'))
try:
    client.create_dataset(dataset) # will fail if dataset already exists
    print('Dataset created')
except:
    print('Dataset already exists')

Dataset already exists


## 2) Create Model

To create a model
1. Use `CREATE MODEL` and provide a destination table for resulting model. Alternatively we can use `CREATE OR REPLACE MODEL` which allows overwriting an existing model.
2. Use `OPTIONS` to specify the model type (linear_reg or logistic_reg). There are many more options [we could specify](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create#model_option_list), such as regularization and learning rate, but we'll accept the defaults.
3. Provide the query which fetches the training data 

**The query will take about two minutes to complete**

In [56]:
%%bq query
CREATE or REPLACE MODEL bqml_taxifare.taxifare_model
OPTIONS(model_type='linear_reg') AS
-- query to fetch training data
SELECT
  (tolls_amount + fare_amount) AS label,
  pickup_datetime,
  pickup_longitude, 
  pickup_latitude, 
  dropoff_longitude, 
  dropoff_latitude
FROM
  `nyc-tlc.yellow.trips`
WHERE
  -- Clean Data
  trip_distance > 0
  AND passenger_count > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  -- repeatable 1/5000th sample
  AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))),5000) = 1

QueryResultsTable job_lHUPXP9VL5HR0KSHyRl443FHTqjB

## Get Training Statistics

Because the query uses a `CREATE MODEL` statement to create a table, you do not see query results. The output is an empty string.

To get the training results we use the [`ML.TRAINING_INFO`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-train) function.

In [57]:
%%bq query
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `bqml_taxifare.taxifare_model`)

training_run,iteration,loss,eval_loss,duration_ms,learning_rate
0,0,67.594182579932,68.80467177368054,6520,


'eval_loss' is reported as mean squared error, so our RMSE is **8.29**. Your results may vary.

## Predict

We predict using `ML.PREDICT`

In [58]:
%%bq query
#standardSQL
SELECT
  predicted_label
FROM
  ML.PREDICT(MODEL `bqml_taxifare.taxifare_model`,
    (
    SELECT
      TIMESTAMP '2014-01-03 00:00:00' as pickup_datetime,
      -73.948 as pickup_longitude, 
      40.779 as pickup_latitude, 
      -73.986 as dropoff_longitude, 
      40.726 as dropoff_latitude
    ))

predicted_label
13.17167269499987


## Recap

The value of BQML is its ease of use:

- We created a model with just two additional lines of SQL
- We never had to move our data out of BigQuery
- We didn't need to use an ML Framework or code, just SQL

There's lots of work going on behind the scenes make this look easy. For example BQML is automatically creating a training/evaluation split, tuning our learning rate,  and one-hot encoding features if neccesary. When we move to TensorFlow these are all things we'll need to do ourselves. 

This notebook was just to inspire usagage of BQML, the current model is actually very poor. We'll prove this in the next lesson by beating it with a simple heuristic. 

We could improve our model considerably with some feature engineering but we'll save that for a future lesson. Also there are additional BQML functions such as `ML.WEIGHTS` and `ML.EVALUATE` that we haven't even explored. If you're interested in learning more about BQML I encourage you to [read the offical docs](https://cloud.google.com/bigquery/docs/bigqueryml).

From here on out we'll focus on pulling data out of BigQuery and building models using TensorFlow, which is more effort but also offers much more flexibility.

Copyright 2019 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.